In [11]:
import psycopg2
import pandas as pd

# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="networkdb",
    user="user",
    password="password"
)

# Lire directement la table dans un DataFrame
df = pd.read_sql("SELECT * FROM reported_flows LIMIT 5;", conn)

# Afficher le DataFrame
print(df)

conn.close()


   id                            flow_key      src_ip      dst_ip  sport  \
0   1  172.30.0.5:80-172.30.0.7:56190-TCP  172.30.0.7  172.30.0.5  56190   
1   2  172.30.0.5:80-172.30.0.7:56178-TCP  172.30.0.7  172.30.0.5  56178   
2   3  172.30.0.4:21-172.30.0.6:59772-TCP  172.30.0.6  172.30.0.4  59772   
3   4  172.30.0.4:21-172.30.0.6:59766-TCP  172.30.0.6  172.30.0.4  59766   
4   5  172.30.0.5:80-172.30.0.7:56170-TCP  172.30.0.7  172.30.0.5  56170   

   dport protocol  total_bytes  pkt_count prediction label_humain  \
0     80      TCP          920         12        Mal       Normal   
1     80      TCP          920         12        Mal       Normal   
2     21      TCP          295         25        Mal       Normal   
3     21      TCP          295         25        Mal       Normal   
4     80      TCP          920         12        Mal       Normal   

                                user_id                reported_at  \
0  fea3f696-8a72-4af5-b774-e0c4e9c942ab 2025-07-12 11:19:1

C:\Users\bensa\AppData\Local\Temp\ipykernel_25484\490516614.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM reported_flows LIMIT 5;", conn)


In [13]:
df

,id,flow_key,src_ip,dst_ip,sport,dport,protocol,total_bytes,pkt_count,prediction,label_humain,user_id,reported_at,flow_data
0,1,172.30.0.5:80-172.30.0.7:56190-TCP,172.30.0.7,172.30.0.5,56190,80,TCP,920,12,Mal,Normal,fea3f696-8a72-4af5-b774-e0c4e9c942ab,2025-07-12 11:19:16.159068,"{""flow_key"": ""172.30.0.5:80-172.30.0.7:56190-T..."
1,2,172.30.0.5:80-172.30.0.7:56178-TCP,172.30.0.7,172.30.0.5,56178,80,TCP,920,12,Mal,Normal,fea3f696-8a72-4af5-b774-e0c4e9c942ab,2025-07-12 11:19:16.170612,"{""flow_key"": ""172.30.0.5:80-172.30.0.7:56178-T..."
2,3,172.30.0.4:21-172.30.0.6:59772-TCP,172.30.0.6,172.30.0.4,59772,21,TCP,295,25,Mal,Normal,fea3f696-8a72-4af5-b774-e0c4e9c942ab,2025-07-12 11:19:16.186894,"{""flow_key"": ""172.30.0.4:21-172.30.0.6:59772-T..."
3,4,172.30.0.4:21-172.30.0.6:59766-TCP,172.30.0.6,172.30.0.4,59766,21,TCP,295,25,Mal,Normal,fea3f696-8a72-4af5-b774-e0c4e9c942ab,2025-07-12 11:19:16.204823,"{""flow_key"": ""172.30.0.4:21-172.30.0.6:59766-T..."
4,5,172.30.0.5:80-172.30.0.7:56170-TCP,172.30.0.7,172.30.0.5,56170,80,TCP,920,12,Mal,Normal,fea3f696-8a72-4af5-b774-e0c4e9c942ab,2025-07-12 11:19:16.217012,"{""flow_key"": ""172.30.0.5:80-172.30.0.7:56170-T..."


In [15]:
import psycopg2
import pandas as pd
import json

# Connexion PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="networkdb",
    user="user",
    password="password"
)

# Charger les données
query = "SELECT flow_data, label_humain FROM reported_flows"
df = pd.read_sql(query, conn)
conn.close()

# Liste des features à extraire
features_to_extract = [
    "total_bytes", "pkt_count", "psh_count", "fwd_bytes", "bwd_bytes",
    "fwd_pkts", "bwd_pkts", "dport", "duration_ms", "flow_pkts_per_s", "fwd_bwd_ratio"
]

# Extraire les features depuis le champ JSON
def extract_features(flow_json):
    try:
        flow = json.loads(flow_json)
        return [flow.get(f, 0) for f in features_to_extract]
    except Exception as e:
        print(f"Erreur parsing JSON: {e}")
        return [0] * len(features_to_extract)

features_df = df["flow_data"].apply(extract_features).apply(pd.Series)
features_df.columns = features_to_extract
features_df["label"] = df["label_humain"].map({"Normal": 0, "Mal": 1})  # ou déjà 0/1

# Résultat final prêt pour entraînement
X = features_df[features_to_extract]
y = features_df["label"]


C:\Users\bensa\AppData\Local\Temp\ipykernel_25484\2606124033.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
